<a href="https://colab.research.google.com/github/Miguelapp10/API_Simpliroute_Urbano/blob/main/API_urbano_simpliroute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import geopandas as gpd
from shapely.geometry import Point, LineString
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from google.colab import files
from google.colab import auth
from google.cloud import bigquery
import warnings

# Suprimir sólo las advertencias de InsecureRequestWarning
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# Información de autenticación
token = ''
base_url_vehicles = 'https://api.simpliroute.com/v1/routes/vehicles/'
base_url_routes = 'https://api.simpliroute.com/v1/routes/routes/'
base_url_visits = 'https://api.simpliroute.com/v1/routes/visits/'

# Encabezados de la solicitud
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Token {token}'
}

# Función para generar un rango de fechas
def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

# Definir el rango de fechas
start_date = datetime.strptime('2024-04-01', '%Y-%m-%d')
end_date = datetime.today()  # Usar la fecha actual para end_date

# Función para obtener datos de una URL en una fecha específica
def fetch_data_for_date(base_url, single_date):
    formatted_date = single_date.strftime('%Y-%m-%d')
    url = f'{base_url}?planned_date={formatted_date}'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        try:
            response_data = response.json()
            if response_data:
                return response_data  # Devolver los datos obtenidos
        except json.JSONDecodeError:
            return []
    return []

# Función para obtener datos de una URL en un rango de fechas usando ThreadPoolExecutor
def get_data_parallel(base_url, start_date, end_date):
    all_data = []
    dates = list(date_range(start_date, end_date))

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_data_for_date, base_url, date) for date in dates]
        for future in futures:
            result = future.result()
            if result:
                all_data.extend(result)

    return all_data

# Obtener datos de ambos endpoints en paralelo
all_vehicles_data = get_data_parallel(base_url_vehicles, start_date, end_date)
all_routes_data = get_data_parallel(base_url_routes, start_date, end_date)
all_visits_data = get_data_parallel(base_url_visits, start_date, end_date)

# Convertir todos los datos a DataFrames de pandas
df_vehicles = pd.json_normalize(all_vehicles_data)
df_routes = pd.json_normalize(all_routes_data)
df_visits = pd.json_normalize(all_visits_data)

# Seleccionar las columnas deseadas de cada DataFrame
df_vehicles_selected = df_vehicles[['id','name',
                                    #'capacity','capacity_2','default_driver','location_start_address',
                                    #'location_start_latitude','location_start_longitude','location_end_address',
                                    #'location_end_latitude','location_end_longitude','created','modified',
                                    'color']].drop_duplicates()
# Renombrar la columna 'id' a 'vehicle'
df_vehicles_selected = df_vehicles_selected.rename(columns={'id': 'vehicle'})

df_routes_selected = df_routes[['id','vehicle','driver','plan','status','planned_date','estimated_time_start',
                                'estimated_time_end','total_duration','total_distance','total_load','total_load_percentage',
                                'location_start_address','location_start_latitude','location_start_longitude',
                                'location_end_address','location_end_latitude','location_end_longitude','start_time',
                                'end_time','created','modified','kilometers','total_visits','latitude_init','longitude_init',
                                'latitude_finish','longitude_finish'
]].drop_duplicates()  # Reemplaza 'other_columns' con las columnas deseadas de df_routes
# Renombrar la columna 'id' a 'vehicle'
df_routes_selected = df_routes_selected.rename(columns={'id': 'route','status':'status_route','created':'created_route','modified':'modified_route'})

# Seleccionar las columnas deseadas
selected_columns = ['id', 'order', 'tracking_id', 'status', 'title', 'address',  'latitude', 'longitude',
                    #'load', 'load_2', 'load_3', 'window_start', 'window_end', 'window_start_2', 'window_end_2', 'duration',
                        'contact_name', 'contact_phone', 'reference', 'notes', 'planned_date', 'route',
                        'route_estimated_time_start', 'estimated_time_arrival', 'estimated_time_departure', 'checkin_time',
                        'checkout_time', 'checkout_latitude', 'checkout_longitude', 'checkout_comment',
                        'checkout_observation',# 'signature','pictures',
                        'created', 'modified', 'eta_predicted',
                        'eta_current', 'driver', 'vehicle', 'on_its_way']
df_visits_selected = df_visits[selected_columns]


# Función para mapear valores de checkout_observation a una nueva columna
def map_observation_to_new_column(observation):
    mapping = {
        '1a1d65aa-d355-45b6-8c3f-3f2295ee4c5a':'Producto no corresponde',
        '56a04e5b-2fc5-42df-b4dc-6ef75d97f63c':'Cliente no quiere identificarse',
        '6084c66c-c720-4136-b20f-e01c80a73378':'Fallas mecánicas',
        '830808ee-2ef6-4c96-973c-751d530ba0f9':'Entregado a conserje',
        '9dc634d3-865f-470b-92c0-14fb63a40637':'Fuera De Horario',
        'e4d21dd5-1107-4c99-a9b7-fae1bac83882':'Entregado a familiar',
        'f97966aa-47f5-4c4d-8d42-1b6df9729157':'Entregado a titular',
        'c505bc38-1215-48bf-9e6b-d78bce3dc2f2':'Cliente Solicito Reprogramación',
        '8c15fdad-2f27-49eb-9967-f4fa94a366b1':'Entregado en tienda',
        '04902a7e-116c-4083-82bb-2849e1928db3':'Cliente Ausente',
        'fe8ac91f-0ead-4d84-b33c-aa094173d32b':'Domicilio Sin Acceso',
        '2fee0a67-e6d8-4742-ac52-f8c4f0382543':'Rechazado Por Cliente',
        '412375a8-cd64-4e8a-81be-5572ca883018':'Producto Dañado'
        }
    return mapping.get(observation, 'Otro')

# Aplicar la función al DataFrame
df_visits_selected['Observaciones'] = df_visits_selected['checkout_observation'].apply(map_observation_to_new_column)

# Unir los DataFrames en el campo común 'vehicle'
df_routes_vehicles = pd.merge(df_routes_selected, df_vehicles_selected, on='vehicle', how='inner')

# Unir los DataFrames en el campo común 'vehicle'
df_visits_routes_vehicles = pd.merge(df_visits_selected, df_routes_vehicles, on=('vehicle','driver' ,'route', 'planned_date'), how='left')

# Guardar el DataFrame resultante en un archivo Excel
#df_visits_routes_vehicles.to_excel('merged_data.xlsx', index=False)
#files.download('merged_data.xlsx')
#print('Datos cruzados y guardados en merged_data.xlsx')
# Configuración de la autenticación para BigQuery
# Crear el DataFrame con los datos proporcionados
# Seleccionar las columnas deseadas
#selected_columns_2 = ['latitude', 'longitude','name','color','planned_date']
#geometry_2 = df_visits_routes_vehicles[selected_columns_2]
# Crear una función para convertir coordenadas en LineString
#def create_linestring(group):
#    coords = list(zip(group['longitude'], group['latitude']))
#    return LineString(coords)
# Aplicar la función para cada grupo y crear un nuevo DataFrame con LineString
#linestrings = geometry_2.groupby(['name', 'color','planned_date']).apply(create_linestring).reset_index()
#linestrings.columns = ['name', 'color','planned_date', 'linestring']
# Convertir el DataFrame a GeoDataFrame
#gdf = gpd.GeoDataFrame(linestrings, geometry='linestring')
# Convert LineStrings to WKT format
#gdf['wkt'] = gdf['linestring'].apply(lambda x: x.wkt)
# Convert WKT to GEOGRAPHY using BigQuery function
#gdf['geography'] = gdf['wkt'].apply(lambda x: f'GEOGRAPHY(ST_GeogFromText("{x}"))')
# Drop the original linestring column
#gdf = gdf.drop(columns=['linestring'])
# Convert planned_date column to datetime format
#gdf['planned_date'] = pd.to_datetime(gdf['planned_date'])

project_id = 'bi-fcom-drmb-local-pe-sbx'
!gcloud config set project {project_id}
# Configuración del dataset y la tabla en BigQuery
dataset_id = 'Pidgeotto_Devolucion'
table_id = 'SimpliRoute_visits_IL'
#table_id_2 = 'SimpliRoute_routes_IL'
# Cliente de BigQuery
client = bigquery.Client(project=project_id)
# Cargar el DataFrame en BigQuery
df_visits_routes_vehicles.to_gbq(destination_table=f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')
# Cargar el DataFrame en BigQuery
#gdf.to_gbq(destination_table=f'{dataset_id}.{table_id_2}', project_id=project_id, if_exists='replace')
# Mostrar el resultado


<ipython-input-1-119f1b9b6d9f>:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_visits_selected['Observaciones'] = df_visits_selected['checkout_observation'].apply(map_observation_to_new_column)


Updated property [core/project].


100%|██████████| 1/1 [00:00<00:00, 7294.44it/s]


In [2]:
import requests
# Configurar cliente de BigQuery
import gzip
import shutil
from datetime import datetime
from google.colab import auth
from google.colab import files # Esto solo es necesario si estás usando Google Colab
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
import os
import gspread as gs
import gspread_dataframe as gd

import concurrent.futures
from google.auth import default

# URL y endpoint para producción
BASE_URL_PROD = "https://api.urbanoexpress.com.pe"
ENDPOINT_PROD = "/api/ws/e-tracking"

# Credenciales de solicitud (reemplazar con tus credenciales reales)
API_KEY = "eb33cb372544c3763cb8ede5bfc18c64e37d5a9496ed43410d99288da8b96fce"

# Suprimir sólo las advertencias de InsecureRequestWarning
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# Autentificación
auth.authenticate_user()
creds, _ = default()
client = bigquery.Client(credentials=creds, project='bi-fcom-drmb-local-pe-sbx')

# Consulta para obtener los números de guía
query = """ select distinct tracking from(
SELECT
  DISTINCT tracking_num as tracking
FROM
  `bi-fcom-drmb-local-pe-sbx.Pidgeotto_Devolucion.Backstore_PE`
WHERE
  creation_dt >= '2024-04-01'
  AND tracking_num LIKE '%WYB%'
UNION ALL
SELECT
  DISTINCT URBANO as tracking
FROM
  `bi-fcom-drmb-local-pe-sbx.Pidgeotto_Devolucion.Backstore_PE`
WHERE
  creation_dt >= '2024-04-01'
  AND URBANO LIKE '%WYB%')
"""
numeros_de_guia = client.query(query).to_dataframe()['tracking'].tolist()

# Función para obtener información de seguimiento
def obtener_informacion_tracking(api_key, guia):
    url = f"https://api.urbanoexpress.com.pe/api/ws/e-tracking"
    headers = {
        'x-api-key': api_key,
        'Content-Type': 'application/json'
    }
    data = {
        "guia_ue": guia,
        "tracking_number": ""
    }
    try:
        response = requests.post(url, headers=headers, json=data, verify=False)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
       # print(f"Error en la solicitud para {guia}: {e}")
        return None

# Función para convertir datos a un DataFrame de pandas
def convertir_a_dataframe(informacion):
    if informacion is None:
        return pd.DataFrame()

    # Crear una lista para almacenar las filas de datos
    rows = []

    # Extraer información principal
    info_principal = {
        #"error_code": informacion.get("error_code", ""),
        #"error_mensaje": informacion.get("error_mensaje", ""),
        #"carrier": informacion.get("carrier", ""),
        #"zone_time": informacion.get("zone_time", ""),
        "guia_ue": informacion.get("guia_ue", ""),
        "tracking_number": informacion.get("tracking_number", ""),
        #"url_tracking": informacion.get("url_tracking", ""),

        "servicio_contrato": informacion.get("servicio", {}).get("contrato", ""),
        "servicio_origen": informacion.get("servicio", {}).get("origen", ""),
        "servicio_admitido": informacion.get("servicio", {}).get("admitido", ""),

        "cliente_codigo": informacion.get("cliente", {}).get("codigo", ""),
        "cliente_nombre": informacion.get("cliente", {}).get("nombre", ""),
        "cliente_telefono": informacion.get("cliente", {}).get("telefono", ""),

        "direccion_entrega": informacion.get("direccion_entrega", {}).get("direccion", ""),
        "direccion_entrega_referencia": informacion.get("direccion_entrega", {}).get("referencia", ""),
        "direccion_entrega_ciudad": informacion.get("direccion_entrega", {}).get("ciudad", ""),
        "direccion_entrega_zona": informacion.get("direccion_entrega", {}).get("zona", ""),
        "direccion_entrega_agencia": informacion.get("direccion_entrega", {}).get("agencia", ""),

        #"estado_actual_codigo": informacion.get("estado_actual", {}).get("codigo", ""),
        "estado_actual_estado": informacion.get("estado_actual", {}).get("estado", ""),
        #"estado_actual_sub_estado": informacion.get("estado_actual", {}).get("sub_estado", ""),
        "estado_actual_Detalle_Estado": informacion.get("estado_actual", {}).get("Detalle_Estado", ""),
        "estado_actual_fecha": informacion.get("estado_actual", {}).get("fecha", "") ,
         "estado_actual_hora":informacion.get("estado_actual", {}).get("hora", "")

        #"datos_envio_contenido": informacion.get("datos_envio", {}).get("contenido", ""),
        #"datos_envio_piezas": informacion.get("datos_envio", {}).get("piezas", ""),
        #"datos_envio_peso": informacion.get("datos_envio", {}).get("peso", ""),
        #"datos_envio_peso_volumen": informacion.get("datos_envio", {}).get("peso_volumen", ""),

        #"remitente_codigo": informacion.get("remitente", {}).get("codigo", ""),
        #"remitente_seller": informacion.get("remitente", {}).get("seller", ""),
        #"remitente_direccion": informacion.get("remitente", {}).get("direccion", ""),
        #"remitente_ciudad": informacion.get("remitente", {}).get("ciudad", ""),

        #"datos_cod_service": informacion.get("datos_cod", {}).get("service", ""),
        #"datos_cod_montos": informacion.get("datos_cod", {}).get("montos", ""),
        #"datos_cod_estado": informacion.get("datos_cod", {}).get("estado", ""),
        #"datos_cod_fecha": informacion.get("datos_cod", {}).get("fecha", ""),

        #"agencia_retiro_agencia": informacion.get("agencia_retiro", {}).get("agencia", ""),
        #"agencia_retiro_horario": informacion.get("agencia_retiro", {}).get("horario", ""),
        #"agencia_retiro_direccion": informacion.get("agencia_retiro", {}).get("direccion", ""),
        #"agencia_retiro_ciudad": informacion.get("agencia_retiro", {}).get("ciudad", ""),
        #"agencia_retiro_latitud": informacion.get("agencia_retiro", {}).get("latitud", ""),
        #"agencia_retiro_longitud": informacion.get("agencia_retiro", {}).get("longitud", "")
    }

    # Extraer información de movimientos
    movimientos = informacion.get("movimiento", [])
    for movimiento in movimientos:
        row = info_principal.copy()  # Copiar información principal
        row.update({
            "movimiento_secuencia": movimiento.get("secuencia", ""),
            "movimiento_codigo": movimiento.get("codigo", ""),
            "movimiento_estado": movimiento.get("estado", ""),
            "movimiento_sub_estado": movimiento.get("sub_estado", ""),
            "movimiento_detalle_estado": movimiento.get("detalle_estado", ""),
            "movimiento_fecha": movimiento.get("fecha", "") + " " + movimiento.get("hora", ""),
            "movimiento_apuntes": movimiento.get("apuntes", ""),
            #"movimiento_agencia": movimiento.get("agencia", ""),
            "movimiento_dir_agencia": movimiento.get("dir_agencia", ""),
            "movimiento_n_visita": movimiento.get("n_visita", "")
        })
        rows.append(row)  # Agregar la fila a la lista

    # Convertir la lista de filas a un DataFrame de pandas
    df = pd.DataFrame(rows)
    # Especificar el formato de las fechas y horas
    date_format = "%Y-%m-%d %H:%M:%S"

    # Convertir la columna de fecha y hora a datetime
    if not df.empty:
        df['movimiento_fecha'] = pd.to_datetime(df['movimiento_fecha'], errors='coerce')
        df['estado_actual_fecha_hora'] = pd.to_datetime(df['estado_actual_fecha'] + " " + df['estado_actual_hora'],  errors='coerce')

        # Eliminar las columnas originales de fecha y hora si ya no son necesarias
        df = df.drop(columns=['estado_actual_fecha', 'estado_actual_hora'])

        # Pivotar los datos para obtener la fecha máxima por movimiento_codigo 'SS' y 'AO'
        #df = df.pivot_table(
        #    index=['guia_ue','tracking_number','servicio_contrato','servicio_origen','servicio_admitido','cliente_codigo',
        #          'cliente_nombre','cliente_telefono','direccion_entrega','direccion_entrega_referencia','direccion_entrega_ciudad',
        #          'direccion_entrega_zona','direccion_entrega_agencia','estado_actual_estado','estado_actual_Detalle_Estado'],
        #    columns='movimiento_codigo',
        #    values='movimiento_fecha',
        #    aggfunc='max'
        #      ).reset_index()

        # Renombrar las columnas pivotadas
        #df.columns.name = None  # Eliminar el nombre de la columna
        #df = df.rename(columns={'SS': 'fecha_SS', 'AO': 'fecha_AO','EN':'fecha_EN'})
    return df
# Procesamiento en paralelo usando ThreadPoolExecutor
def procesar_tracking(api_key, numeros_de_guia, max_workers=30):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_guia = {executor.submit(obtener_informacion_tracking, api_key, guia): guia for guia in numeros_de_guia}
        resultados = []
        for future in concurrent.futures.as_completed(future_to_guia):
            guia = future_to_guia[future]
            try:
                resultado = future.result()
                if resultado:  # Solo agregar resultados válidos
                    resultados.append(resultado)
            except Exception as e:
                print(f"Error al procesar la información de tracking para {guia}: {e}")
    return resultados

resultados = procesar_tracking(API_KEY, numeros_de_guia, max_workers=30)

# Convertir resultados a DataFrame
df_total = pd.concat([convertir_a_dataframe(r) for r in resultados if r], ignore_index=True)

# Guarda el DataFrame como un archivo Excel
#archivo_excel = 'informacion_tracking_total.xlsx'
#df_total.to_excel(archivo_excel, index=False)

# Comprimir el archivo Excel utilizando gzip
#archivo_comprimido = 'informacion_tracking_total.xlsx.gz'
#with open(archivo_excel, 'rb') as f_in:
#    with gzip.open(archivo_comprimido, 'wb') as f_out:
#        shutil.copyfileobj(f_in, f_out)

# Descargar el archivo Excel comprimido
#files.download(archivo_comprimido)

project_id = 'bi-fcom-drmb-local-pe-sbx'
!gcloud config set project {project_id}
# Configuración del dataset y la tabla en BigQuery
dataset_id = 'Pidgeotto_Devolucion'
table_id = 'URBANO_IL'

# Cliente de BigQuery
client = bigquery.Client(project=project_id)
# Cargar el DataFrame en BigQuery
df_total.to_gbq(destination_table=f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')


<ipython-input-2-d25c0c5f088c>:159: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['movimiento_fecha'] = pd.to_datetime(df['movimiento_fecha'], errors='coerce')
<ipython-input-2-d25c0c5f088c>:159: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['movimiento_fecha'] = pd.to_datetime(df['movimiento_fecha'], errors='coerce')
<ipython-input-2-d25c0c5f088c>:159: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['movimiento_fecha'] = pd.to_datetime(df['movimiento_fecha'], errors='coerce')
<ipython-input-2-d25c0c5f088c>:160: UserWarning: Could not infer format, so each element wi

Updated property [core/project].


100%|██████████| 1/1 [00:00<00:00, 576.30it/s]
